# Data Cleaning and De-identification Pipeline

This notebook processes raw Excel files containing conversations.  
The goal is to:
1. Load `.xlsx` files from a folder.  
2. Clean and de-identify sensitive information using Azure OpenAI.  
3. Save the processed conversations as `.json` files.  

In [1]:
import os
import asyncio
import pandas as pd
import json
from langdetect import detect
from openai import AzureOpenAI

### 1. Define input and output directories

We set the folder where the raw Excel files are stored, and also create an output
directory (`cleaned_json1`) where the processed JSON files will be saved.

In [9]:
folderpath = "../raw data"
output_dir = "cleaned_json1"
os.makedirs(output_dir, exist_ok=True)

### 2. Initialize the Azure OpenAI client

We configure the client with the API key, version, and endpoint.  
This client will be used later to call the `chat.completions` endpoint for text cleaning.

In [10]:
client = AzureOpenAI(
    api_key="YOUR-API-KEY",
    api_version="2024-02-15-preview",
    azure_endpoint="https://fallbackmodel.openai.azure.com/"
)

### 3. Define text cleaning function

This function sends the text to the Azure OpenAI model with a detailed prompt.  
The model returns the cleaned and de-identified version of the input.

In [11]:
def clean_and_deidentify_text(text):
    prompt = f"""
    You are a data cleaning assistant. When given a text, you will return ONLY the cleaned and de-identified version of the text, with no extra labels, no explanations, and no headers.
    Replace names, phone numbers, and locations with <PERSON>, <PHONE_NUMBER>, <LOCATION>, etc.
    Replace email addresses with <EMAIL> and URLs or web links with <URL>.
    Replace personal IDs or account numbers with <ID_NUMBER> or <ACCOUNT_NUMBER>.
    Replace dates and times with <DATE> or <TIME> in a consistent format.
    Replace emojis with descriptive placeholders in the format (:happy), (:sad), (:thumbs_up), etc., so the meaning remains clear.
    Remove noise like excessive punctuation and irrelevant words.
    Normalize text casing to sentence case, remove extra spaces or tabs, and replace double newlines with single newlines.
    Correct obvious spelling mistakes and fix simple grammar errors where possible.
    
    Original Text:
    {text}
    """
    response = client.chat.completions.create(
        model="gpt-4o-fallback",
        messages=[
            {"role": "system", "content": "You are a helpful assistant for data cleaning."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5,
        max_tokens=2000
    )
    return response.choices[0].message.content.strip()

### 4. Process a single conversation

This function takes a DataFrame representing a conversation and a conversation ID.  
It iterates over each row, cleans the text, and builds a list of messages.  
It also detects the overall language of the conversation.

In [12]:
def process_conversation(conversation_df, conversation_id):

    detected_language = "unknown"

    cleaned_messages = []
    all_text = ""

    for _, row in conversation_df.iterrows():
        role = str(row["role"]).strip().lower()
        text = str(row["text"]).strip()

        try:
            cleaned_text = clean_and_deidentify_text(text)
        except Exception:
            cleaned_text = "<CONTENT_FILTERED>"

        all_text += cleaned_text + "\n"

        cleaned_messages.append({
            "role": "user" if role == "user" else "bot",
            "content": cleaned_text
        })

    try:
        detected_language = detect(all_text)
    except:
        detected_language = "unknown"

    return {
        "conversation_id": conversation_id,
        "messages": cleaned_messages,
        "language": detected_language
    }

### 5. Process a single file

This asynchronous function:
1. Reads the Excel file into a pandas DataFrame.
2. Processes the conversation using `process_conversation`.
3. Saves the result as a JSON file in the output folder.

In [18]:
async def process_file(file_path, output_folder):
    try:
        df = pd.read_excel(file_path)

        base_filename = os.path.splitext(os.path.basename(file_path))[0]
        conversation_json = process_conversation(df, base_filename)

        output_file_path = os.path.join(output_folder, f"{base_filename}.json")
        with open(output_file_path, 'w', encoding='utf-8') as json_file:
            json.dump(conversation_json, json_file, indent=4, ensure_ascii=False)

        print(f"{base_filename}.xlsx processed")
    except Exception as e:
        print(f"Error in file {file_path}: {e}")

### 6. Manage concurrency with a semaphore

We limit the number of concurrent tasks to 10 in order not to overload the API or system resources.

In [17]:
semaphore = asyncio.Semaphore(10)

### 7. Main function

The `main` function:
1. Lists all `.xlsx` files in the input folder.
2. Defines an async wrapper `limited_process` that respects the semaphore.
3. Creates tasks for all Excel files and runs them concurrently.
4. Prints a completion message after all tasks finish.

In [19]:
async def main():
    files = os.listdir(folderpath)
    xlsx_files = [f for f in files if f.endswith(".xlsx")]

    async def limited_process(file):
        async with semaphore:
            file_path = os.path.join(folderpath, file)
            await process_file(file_path, output_dir)

    tasks = [limited_process(file) for file in xlsx_files]
    await asyncio.gather(*tasks)

    print(f"\n Process complete")

### 8. Run the pipeline

Finally, we run the `main()` function to start processing the files.

In [20]:
await main()

conversation1.xlsx processed
conversation10.xlsx processed
conversation2.xlsx processed
conversation3.xlsx processed
conversation4.xlsx processed
conversation5.xlsx processed
conversation6.xlsx processed
conversation7.xlsx processed
conversation8.xlsx processed
conversation9.xlsx processed

 Process complete
